# 60. 単語ベクトルの読み込みと表示

In [2]:
import gensim

unable to import 'smart_open.gcs', disabling that module


In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
model['United_States']

array([-3.61328125e-02, -4.83398438e-02,  2.35351562e-01,  1.74804688e-01,
       -1.46484375e-01, -7.42187500e-02, -1.01562500e-01, -7.71484375e-02,
        1.09375000e-01, -5.71289062e-02, -1.48437500e-01, -6.00585938e-02,
        1.74804688e-01, -7.71484375e-02,  2.58789062e-02, -7.66601562e-02,
       -3.80859375e-02,  1.35742188e-01,  3.75976562e-02, -4.19921875e-02,
       -3.56445312e-02,  5.34667969e-02,  3.68118286e-04, -1.66992188e-01,
       -1.17187500e-01,  1.41601562e-01, -1.69921875e-01, -6.49414062e-02,
       -1.66992188e-01,  1.00585938e-01,  1.15722656e-01, -2.18750000e-01,
       -9.86328125e-02, -2.56347656e-02,  1.23046875e-01, -3.54003906e-02,
       -1.58203125e-01, -1.60156250e-01,  2.94189453e-02,  8.15429688e-02,
        6.88476562e-02,  1.87500000e-01,  6.49414062e-02,  1.15234375e-01,
       -2.27050781e-02,  3.32031250e-01, -3.27148438e-02,  1.77734375e-01,
       -2.08007812e-01,  4.54101562e-02, -1.23901367e-02,  1.19628906e-01,
        7.44628906e-03, -

# 61. 単語の類似度

In [12]:
model.similarity('United_States', 'U.S.')

0.73107743

# 62. 類似度の高い単語10件

In [13]:
model.most_similar('United_States')

[('Unites_States', 0.7877248525619507),
 ('Untied_States', 0.7541370391845703),
 ('United_Sates', 0.74007248878479),
 ('U.S.', 0.7310774326324463),
 ('theUnited_States', 0.6404393911361694),
 ('America', 0.6178410053253174),
 ('UnitedStates', 0.6167312264442444),
 ('Europe', 0.6132988929748535),
 ('countries', 0.6044804453849792),
 ('Canada', 0.6019070148468018)]

# 63. 加法構成性によるアナロジー

In [14]:
model.most_similar(positive = ['Spain', 'Athens'], negative = ['Madrid'])

[('Greece', 0.6898480653762817),
 ('Aristeidis_Grigoriadis', 0.5606849193572998),
 ('Ioannis_Drymonakos', 0.555290937423706),
 ('Greeks', 0.5450686812400818),
 ('Ioannis_Christou', 0.5400862693786621),
 ('Hrysopiyi_Devetzi', 0.5248445272445679),
 ('Heraklio', 0.5207759141921997),
 ('Athens_Greece', 0.516880989074707),
 ('Lithuania', 0.5166865587234497),
 ('Iraklion', 0.5146791338920593)]

# 64. アナロジーデータでの実験

In [4]:
with open('./data/questions-words.txt', mode = 'r+') as f:
    in_file_data = f.readlines()
    for idx, data in enumerate(in_file_data):
        if data[0] == ':': continue
        out_line = data.strip('\n')
        splited_data = data.split()
        most_similar_result = model.most_similar(positive = [splited_data[1], splited_data[2]], negative = [splited_data[0]], topn = 1)
        out_line = ' '.join([out_line, most_similar_result[0][0], str(most_similar_result[0][1])])
        in_file_data[idx] = out_line + '\n'
    f.writelines(out_line)

0
507
5032
5899
8367
8874
9867
10680
12013
13136
14193
15793
17354
18687


UnsupportedOperation: not writable

In [16]:
#for idx, data in enumerate(in_file_data):
#    data = data.strip('\n')
#    data += '\n'
#in_file_data[idx] = data

In [18]:
#with open('./data/questions-words.txt', mode = 'w') as out_file:
#    out_file.writelines(in_file_data)

# 65. アナロジータスクでの正解率

In [47]:
from itertools import islice

def calc_correct_answer_rate(data_iter):
    num_correct = 0
    num_data = 0
    for data in data_iter:
        if data[0] == ':': continue
        splited_data = data.split() 
        if splited_data[3] == splited_data[4]: num_correct += 1
        num_data += 1
    return num_correct / num_data
    

with open('./data/questions-words.txt', mode = 'r') as in_file:
    semantic_analogy_size = 8874
    semantic_analogy_iter = islice(in_file, semantic_analogy_size)
    correct_answer_rate_semantic_analogy = calc_correct_answer_rate(semantic_analogy_iter)
    print(f'correct answer rate(semantic analogy) : {correct_answer_rate_semantic_analogy}')
    syntactic_analogy_size = 10684
    syntactic_analogy_iter = islice(in_file, syntactic_analogy_size)
    correct_answer_rate_syntactic_analogy = calc_correct_answer_rate(syntactic_analogy_iter)
    print(f'correct answer rate(syntactic_analogy): {correct_answer_rate_syntactic_analogy}') 

correct answer rate(semantic analogy) : 0.7308602999210734
correct answer rate(syntactic_analogy): 0.7400468384074942


# 66. WordSimilarity-353での評価

In [124]:
import pandas as pd
import pprint
from scipy.stats import spearmanr

In [91]:
col_names = ['Word1', 'Word2', 'Human']
df = pd.read_csv("./data/wordsim353/combined.csv",encoding = "UTF-8", names = col_names)
row_size, _ = df.shape
data_list = [{} for _ in range(row_size - 1)]
for data_name in col_names:
    for idx, frame in enumerate(df[1:][data_name]):
        data_list[idx][data_name] = frame

In [131]:
similarity = list()
human_score = list()
for idx, data in enumerate(data_list):
    word1 = data['Word1']
    word2 = data['Word2']
    similarity.append([word1, word2, model.similarity(word1, word2), idx])
    human_score.append([word1, word2, data['Human'], idx])
sorted_similarity = sorted(similarity, reverse = True, key = lambda x: x[2])
sorted_human_score = sorted(human_score, reverse = True, key = lambda x: x[2])

In [130]:
pprint.pprint(sorted_similarity[:10])
pprint.pprint(sorted_human_score[:10])

[['tiger', 'tiger', 1.0, 2],
 ['Harvard', 'Yale', 0.7817695, 209],
 ['man', 'woman', 0.76640123, 306],
 ['football', 'soccer', 0.73135483, 42],
 ['mile', 'kilometer', 0.7258478, 170],
 ['vodka', 'brandy', 0.6881493, 59],
 ['journey', 'voyage', 0.68308526, 68],
 ['football', 'basketball', 0.66824687, 43],
 ['type', 'kind', 0.66664124, 236],
 ['championship', 'tournament', 0.66553164, 296]]
[['fuck', 'sex', '9.44', 40],
 ['journey', 'voyage', '9.29', 68],
 ['midday', 'noon', '9.29', 73],
 ['dollar', 'buck', '9.22', 266],
 ['money', 'cash', '9.15', 31],
 ['coast', 'shore', '9.10', 70],
 ['money', 'cash', '9.08', 97],
 ['money', 'currency', '9.04', 98],
 ['football', 'soccer', '9.03', 42],
 ['magician', 'wizard', '9.02', 72]]


In [141]:
rank_similarity = [0] * len(sorted_similarity)
for rank, data in enumerate(sorted_similarity):
    rank_similarity[data[3]] = rank + 1
    
rank_human_score = [0] * len(sorted_human_score)
for rank, data in enumerate(sorted_human_score):
    rank_human_score[data[3]] = rank + 1

correlation, pvalue = spearmanr(rank_similarity, rank_human_score)

In [142]:
print(correlation, pvalue)

0.6848782939104398 3.449094093160346e-50


# 67. k-meansクラスタリング

In [150]:
!cut -f 1 < ./data/country-data-wiki.txt > ./work/country-names.txt

In [190]:
country_list = list()
country_vec = list()
with open('./work/country-names.txt', mode = 'r') as in_file:
    for data in in_file:
        data = data.strip()
        data = data.replace(' ', '_')
        try:
            country_vec.append(model[data])
            country_list.append(data)
        except KeyError:
            pass

In [193]:
print(len(country_list))

197


In [183]:
import numpy as np
from sklearn.cluster import KMeans

In [372]:
result_pred = KMeans(n_clusters = 5).fit_predict(country_vec)
for (country, pred) in zip(country_list, result_pred):
    print(country, pred)

Afghanistan 4
Åland_Islands 0
Albania 0
Algeria 3
Andorra 0
Angola 3
Anguilla 1
Antarctica 1
Argentina 2
Armenia 0
Aruba 2
Australia 4
Austria 0
Azerbaijan 0
Bahamas 1
Bahrain 4
Bangladesh 4
Barbados 1
Belarus 0
Belgium 0
Belize 1
Benin 3
Bermuda 1
Bhutan 4
Botswana 3
Bouvet_Island 0
Brazil 2
Brunei_Darussalam 4
Bulgaria 0
Burkina_Faso 3
Burundi 3
Cabo_Verde 1
Cambodia 4
Cameroon 3
Canada 0
Cayman_Islands 1
Chad 3
Chile 2
China 4
Colombia 2
Comoros 3
Congo 3
Cook_Islands 1
Costa_Rica 2
Croatia 0
Cuba 2
Curaçao 1
Cyprus 0
Czechia 0
Denmark 0
Djibouti 3
Dominica 1
Dominican_Republic 2
Ecuador 2
Egypt 4
El_Salvador 2
Equatorial_Guinea 3
Eritrea 3
Estonia 0
Ethiopia 3
Faroe_Islands 0
Fiji 1
Finland 0
France 0
French_Guiana 2
French_Polynesia 1
Gabon 3
Gambia 3
Georgia 0
Germany 0
Ghana 3
Gibraltar 1
Greece 0
Greenland 0
Grenada 1
Guadeloupe 1
Guam 1
Guatemala 2
Guernsey 1
Guinea 3
Guyana 1
Haiti 2
Holy_See 0
Honduras 2
Hong_Kong 4
Hungary 0
Iceland 0
India 4
Indonesia 4
Iraq 4
Ireland 0
Is

In [254]:
import numpy as np
from sklearn.base import BaseEstimator, ClusterMixin
from random import randint

class myKMeans(BaseEstimator, ClusterMixin):

    def __init__(self, n_clusters=2, max_iter=10, random_state=False, init='random',
               algorithm = 'full', n_init=10, n_jobs=1, tol=1e-4, verbose=False):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.init = init #実装してない
        self.algorithm = algorithm #実装してない
        self.n_init = n_init
        self.tol = tol
        self.n_jobs = n_jobs #実装してない
        self.verbose = verbose #実装してない

    def fit(self, X, y=None):
        self.inertia_ = np.inf
        for iter in range(self.n_init):
            if self.random_state==False:
                np.random.seed(seed=1)
            cluster_centers = None
            n_iter = None
            inertia = np.inf
            #初期化はラベルをランダムで振り分ける(実装によっては重心を要素の中から決める方法もある)
            now_pred = np.random.randint(0, self.n_clusters, len(X))
            for _ in range(self.max_iter):
                cluster_centers = np.array([X[now_pred == i].mean(axis=0)
                    for i in range(self.n_clusters)])
                new_pred = np.array([
                    np.array([
                        self.distance_(p, centroid)
                            for centroid in cluster_centers
                    ]).argmin()
                    for p in X
                ])
                new_inertia = np.sum(
                    np.array([
                        np.array([
                            self.distance_(p, centroid)
                                for centroid in cluster_centers
                            ]).min()
                        for p in X
                    ])
                )

                if abs(new_inertia - inertia) < self.tol:
                    n_iter = _
                    break
                inertia = new_inertia
                now_pred = new_pred

            if self.inertia_ > inertia:
                self.n_iter_ = n_iter
                self.inertia_ = inertia
                self.labels_ = now_pred
                self.cluster_centers_ = cluster_centers
        return self.labels_

    def distance_(self, a, b):
        return np.sum((a - b) ** 2)

In [276]:
my_kmeans = myKMeans(n_clusters = 5)
my_result_pred = my_kmeans.fit_predict(np.array(country_vec))

# 68. Ward法によるクラスタリング

In [349]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

In [350]:
Z = linkage(country_vec, method = 'ward', metric='euclidean')

In [361]:
plt.figure()
dendrogram(Z, labels = country_list)
plt.show()
plt.savefig('./work/country_dendrogram')

<IPython.core.display.Javascript object>

In [375]:
pred_ward = []
print(Z)

[[ 25.         156.           1.17092872   2.        ]
 [ 17.          93.           1.70162888   2.        ]
 [ 24.         195.           1.81941397   2.        ]
 [ 62.         175.           1.86109447   2.        ]
 [ 11.         134.           1.92832628   2.        ]
 [ 85.         154.           1.94213775   2.        ]
 [ 49.         140.           1.9476361    2.        ]
 [ 12.          69.           1.9799525    2.        ]
 [ 55.          77.           1.98105232   2.        ]
 [ 58.         102.           1.98689824   2.        ]
 [ 18.         187.           2.00300536   2.        ]
 [ 19.         133.           2.03082807   2.        ]
 [ 84.         165.           2.06701808   2.        ]
 [ 23.         132.           2.0831231    2.        ]
 [ 44.         167.           2.10106573   2.        ]
 [  9.          13.           2.11393916   2.        ]
 [  8.          26.           2.11520131   2.        ]
 [ 98.         186.           2.13647304   2.        ]
 [ 53.    

# 69. t-SNEによる可視化

In [362]:
from sklearn.manifold import TSNE

In [369]:
X_embedded = TSNE(n_components=2).fit_transform(country_vec)
X_embedded.shape
print(X_embedded)

[[-4.14389944e+00  1.99217880e+00]
 [ 1.20285118e+00  1.63771975e+00]
 [-1.90847027e+00  8.84716606e+00]
 [-9.21663570e+00  1.87624848e+00]
 [ 2.36376071e+00  1.24727011e+01]
 [-1.56410475e+01  5.31350315e-01]
 [ 6.82165575e+00 -4.31433105e+00]
 [-2.12653220e-01  2.54532027e+00]
 [ 8.43560600e+00  4.37542772e+00]
 [-4.23336887e+00  9.46137428e+00]
 [ 7.82240152e+00 -9.81796980e-01]
 [ 3.41382176e-01 -8.18800831e+00]
 [ 1.17223346e+00  7.40488958e+00]
 [-4.73896599e+00  9.84598255e+00]
 [ 6.93049622e+00 -5.61127663e+00]
 [-4.43992472e+00 -1.69648266e+00]
 [-4.89853334e+00 -8.50235558e+00]
 [ 7.72867680e+00 -4.57660103e+00]
 [-3.78326750e+00  1.12962446e+01]
 [ 2.87643886e+00  5.89982796e+00]
 [ 7.88897514e+00 -5.73245573e+00]
 [-1.33833084e+01  4.02068049e-01]
 [ 5.63427544e+00 -4.75026941e+00]
 [-7.05438185e+00 -7.55463552e+00]
 [-1.57282095e+01 -2.51771498e+00]
 [ 1.24696136e+00  6.84367537e-01]
 [ 7.91930819e+00  4.06522131e+00]
 [-2.34338021e+00 -7.19730425e+00]
 [-1.99383497e+00  9

In [374]:
plt.figure()
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c = result_pred)
plt.colorbar()
plt.savefig('./work/country_t-sne')

<IPython.core.display.Javascript object>